##### Appendix B: Problem 7

Given information - Components names, Molecular Weights & Molar Fractions

*Note that there is a misprint in the Monograph equations for the evaluation of Lohrenz-Bray-Clark viscosity - corrected below in the 'squiggle_ti' list

In [2]:
# From table B-9 (same data as provided for Problem 6)
names = ['C1', 'C2', 'C3', 'iC4', 'nC4', 'iC5', 'nC5', 'C6', 'C7+']
z = [0.875, 0.083, 0.021, 0.006, 0.008, 0.003, 0.002, 0.001, 0.001]
R = 10.73146 # Universal gas constant, psia-ft3/R-lbm mol

# From Table 10
M = [16.04, 30.07, 44.09, 58.12, 58.12, 72.15, 72.15, 86.17, 114.0]
pc = [667.8, 707.8, 616.3, 529.1, 550.7, 490.4, 488.6, 436.9, 360.6]
tc = [343, 549.8, 665.7, 734.7, 765.3, 828.3, 845.4, 913.4, 1023.9]

# From table B-11 (also available from table A-1A)
vc = [1.59, 2.37, 3.25, 4.208, 4.08, 4.899, 4.87, 5.929, 7.882]
zc = [0.2884, 0.2843, 0.2804, 0.2824, 0.2736, 0.2701, 0.2623, 0.2643, 0.2587]

In [51]:
# Solution
import math
ncomps = len(z)
t = 160 # deg F
p = 2000+14.7 #psia

# Molar weighted critical temperature
ztc = [z[i]*tc[i] for i in range(ncomps)]
Tpc = sum(ztc)
Tpr = (t + 460)/Tpc

# Molar weighted Zc
zzc = [z[i]*zc[i] for i in range(ncomps)]
Zpc = sum(zzc)

# Molar weighted vc
zvc = [z[i]*vc[i] for i in range(ncomps)]
vpc = sum(zvc)

# Ppc calculated from Eq 3.67
Ppc = R * Tpc * Zpc / vpc
Ppr = p / Ppc

# Average MW
zM = [z[i]*M[i] for i in range(ncomps)]
Mg = sum(zM)

# Lucas correlation
squiggle = 9490*(Tpc/(Mg**3*Ppc**4))**(1/6)
ugsc_squiggle = (0.807*Tpr**0.618-0.357*math.exp(-0.449*Tpr)+0.340*math.exp(-4.058*Tpr)+0.018)
ugsc = ugsc_squiggle / squiggle
A1 = 1.245e-3*math.exp(5.1726*Tpr**-0.3286)/Tpr
A2 = A1 * (1.6553*Tpr - 1.2723)
A3 = 0.4489*math.exp(3.0578*Tpr**-37.7332)/Tpr
A4 = 1.7368*math.exp(2.231*Tpr**-7.6351)/Tpr
A5 = 0.9425*math.exp(-0.1853*Tpr**0.4489)
ug_on_ugsc = 1 + A1*Ppr**1.3088 / (A2*Ppr**A5 + (1 + A3*Ppr**A4)**-1)
ug_lucas = ug_on_ugsc * ugsc
print('              Lucas Correlation Vicosity:',round(ug_lucas,4), 'cP' )

# Lohrenz-Bray-Clark Correlation
# Start with (re)calculating gas density using Hall-Yarborough Z factor from Problem 6
from scipy.optimize import fsolve
def fy(y, *args):
    alpha, Pr, t = args
    return -alpha * Pr + (y + y ** 2 + y ** 3 - y ** 4) / (1 - y) ** 3 - (14.76 * t - 9.76 * t ** 2 + 4.58 * t ** 3) * y ** 2 + (90.7 * t - 242.2 * t ** 2 + 42.4 * t ** 3) * y ** (2.18 + 2.82 * t)
    
def Zfac(Tr, Pr):
    t = 1 / Tr
    alpha = 0.06125 * t * math.exp(-1.2 * (1 - t) ** 2)
    args = (alpha, Pr, t)
    y = 0.001
    y = fsolve(fy, y, args=args)
    return alpha * Pr / y

Z = Zfac(Tpr, Ppr)[0]
rho_g = (p*Mg)/(Z*R*(t+460))

rho_pr = (rho_g/Mg)*vpc # Eq 3.1333c
squiggle_t = 5.35*(Tpc/(Mg**3*Ppc**4))**(1/6)
Tr = [(t+460)/tc[i] for i in range(ncomps)] # Component reduced temperatures
squiggle_ti = [5.35*(tc[i]/(M[i]**3*pc[i]**4))**(1/6) for i in range(ncomps)] # ** Note ** This equation is misprinted in the Monograph, the M**3 term should be in the denominator (like Eq 3.133b)
u = [0]*ncomps
for i in range(ncomps):
    if Tr[i] <= 1.5:
        u[i] = 34e-5*Tr[i]**0.94 / squiggle_ti[i]
    else:
        u[i] = 17.78e-5*(4.58*Tr[i]-1.67)**(5/8) / squiggle_ti[i]
    
u0 = sum(z[i]*u[i]*M[i]**0.5 for i in range(ncomps)) / sum(z[i]*M[i]**0.5 for i in range(ncomps)) # ugsc
u_lohrenz = ((0.1023+0.023364*rho_pr+0.058533*rho_pr**2+-0.040758*rho_pr**3 + 0.0093324*rho_pr**4)**4 - 1e-4)/squiggle_t + u0

print('Lohrenz-Bray-Clark Correlation Viscosity:',round(u_lohrenz,4), 'cP' )


              Lucas Correlation Vicosity: 0.0167 cP
Lohrenz-Bray-Clark Correlation Viscosity: 0.0166 cP
